# node.js 2


## Introduction to Buffers

A data buffer (or just buffer) is a region of a physical memory storage used to temporarily store data while it is being moved from one place to another. Typically, the data is stored in a buffer as it is retrieved from an input device (such as a microphone) or just before it is sent to an output device (such as speakers). However, a buffer may be used when moving data between processes within a computer. Buffers can be implemented in a fixed memory location in hardware—or by using a virtual data buffer in software, pointing at a location in the physical memory. A majority of buffers are implemented in software, which typically use the faster RAM to store temporary data, due to the much faster access time compared with hard disk drives. Buffers are typically used when there is a difference between the rate at which data is received and the rate at which it can be processed, or in the case that these rates are variable, for example in a printer spooler or in online video streaming.

A buffer often adjusts timing by implementing a queue (or FIFO) algorithm in memory, simultaneously writing data into the queue at one rate and reading it at another rate.

You can conceptualize a buffer as an array of integer or floating point numbers of various lengths, or as a binary string. Unlike higher-level data structures like arrays, a buffer is not resizable.

It corresponds roughly to:

- char* or char[] in C/C++
- byte[] in Java
- A mutable bytes or a non-resizable bytearray in Python
- Strings in php if they were mutable

## Buffer class in node.js

Pure JavaScript is Unicode friendly, but it is not so for binary data. When programmatically interfacing with TCP streams or the file system, it's often necessary to handle octet (byte) streams (i.e. binary streams). Node provides the `Buffer` class which provides instances to store raw binary data similar to an array of integers but corresponds to a raw memory allocation outside the V8 heap. The `Buffer` class is a global class in `node.js` that can be accessed in an application without importing the buffer module.

There are 3 different ways of creating a Buffer instance in `node.js`:

In [7]:
var buffer1 = new Buffer(10);
var buffer2 = new Buffer([10, 20, 30, 40, 50]);
var buffer3 = new Buffer("Otago Polytechnic", "utf-8");

undefined

Though "utf8" is the default encoding, you can use any of the following encodings "ascii", "utf8", "utf16le", "ucs2", "base64" or "hex".

Writing to buffers is relatively straightforward:

In [8]:
buffer1.write("abcdef");

6

for reading from a buffer you need to specify the encoding to use as well as the beginning index to start reading and the end index to and reading :

In [9]:
buffer1.toString('ascii',0,3)

'abc'

Buffers can also be easily converted to a JSON data structure:

In [10]:
buffer1.toJSON()

{ type: 'Buffer',
  data: [ 97, 98, 99, 100, 101, 102, 0, 0, 136, 238 ] }

## The file system

One of the main differences between client-side JavaScript analogy and `node.js` is that `node.js` allows us to interact with the file system.

The `fs` module can be used to list files and directories, create files and directories, stream files, write files, read files, modify file permissions or just about anything that you need to be able to do with the file system.

We have already seen how to read directories and files, but let's recap:

In [ ]:
var fs = require("fs");

//reading the content of current folder
fs.readdir('./', function(err, files) {

    if (err) {
        throw err;
    }

    console.log(files);

});

In [13]:
var fs = require("fs");

// reading the content of file named 'input.txt'
fs.readFile('./input.txt', "UTF-8", function(err, contents) {

    console.log(contents);

});

undefined

I am the content of file input.txt


Another feature of the file system module is the ability to create new files, to write text or binary content to those files, or to append text or binary content to an existing file.

In [15]:
var fs = require("fs");

var md = `

Sample Markdown Title
=====================

Sample subtitle
----------------

* point
* point
* point

`;

fs.writeFile("sample.md", md.trim(), function(err) {

    console.log("File Created");

});

fs.appendFile("sample.md", "\n\noops, I forgot something");


undefined

File Created


the FS module also gives us tools for creating directories.

In [13]:
var fs = require("fs");
fs.mkdir("lib", function(err) {

    if (err) {
        console.log(err);
    } else {
        console.log("Directory Created");
    }

});

undefined

Directory Created


removing directories is also trivial:

In [14]:
var fs = require("fs");
fs.rmdir("./lib", function(err) {

    if (err) {
        throw err;
    }

    console.log("lib directory removed");

});

undefined

lib directory removed


You may need to move, rename, or remove a file, and the fs module has methods for that as well.

In [16]:
var fs = require("fs");

//the rename method can be used both for renaming a file or for moving it to a different location
fs.rename("./sample.md", "./sample2.md", function(err) {

    if (err) {
        console.log(err);
    } else {
        console.log("sample.md renamed/moved successfully");
    }

});

undefined

sample.md renamed/moved successfully


In [17]:
var fs = require("fs");
fs.unlink("sample2.md", function(err) {

    if (err) {
        console.log(err);
    } else {
        console.log("sample2.md removed");
    }

});

undefined

sample2.md removed


## Streams

Streams give us a way to asynchronously handle continuous data flows. Understanding how streams work will dramatically improve the way your application handles large data. Streams in `Node.js` are implementations of the underlying abstract Streams interface and we've already been using them because process standard input and process standard output both implement the Stream interface.

Simply put, a stream is nothing but an EventEmitter and implements some specials methods. Depending on the methods implemented, a stream becomes readable, like stdin, writeable like standard output, or duplex, which means they are both readable and writeable. Streams can work with binary data or data encoded in a text format like UTF-8. 

### Readable streams

To illustrate the usefulness of the streams, let's think about a program that needs to read a **very, very** looooooong text file. the problem with a traditional approach, is that the `readFile` method will wait until the entire file is read before invoking the callback function and passing the file contents. This is fine for small and medium-size files but not optimal for very large files. Such an approach will buffer the entire file in one variable, requiring tones of memory and also creating inappropriate latency in the application. With a stream, as opposed to waiting for the entire file to be read, we can use the stream to start receiving small chunks of data from the file as it is being read. This could be very useful to stream or video files for instance. Let's see how to implement a readable stream in `node.js`.

First, let's illustrate reading a not so big file (6 MB) using the traditional approach of reading the entire content in a single buffer and waiting for the file reading to complete before handling controll to the callback function.

In [40]:
var fs = require("fs");

// reading the entire content of longTextFile.txt will take a few seconds
fs.readFile('./longTextFile.txt', "UTF-8", function(err, contents) {

    console.log("Done!");
    /*We can console.log the content in case we want to print out the contents of the file, which I don't 
    recommend because it is an entire book
    and it will take the console  a while to print it all out
    */
    //console.log(contents); 

});

undefined

Done!


Instead, let's read the content of the file using a readable stream and handling the data chunk by chunk. Obviously, for a file that it's only 6 MB in size, the streaming approach does not make a lot of sense. But for very large files (>1GB) it could be very useful. Node.js streams are event emitters so you can listen to its events to monitor the data being transmitted:

In [9]:
var fs = require("fs");

var stream = fs.createReadStream("./longTextFile.txt", "UTF-8");

var data = "";

//this method executes only on the 1st data event
stream.once("data", function() {
    console.log("\n\n\n");
    console.log("Started Reading File");
    console.log("\n\n\n");
});

stream.on("data", function(chunk) {
    process.stdout.write(`  chunk: ${chunk.length} |`);
    data += chunk;
    //console.log(chunk); //in case we want to print out the contents of the chunk
}); 

stream.on("end", function() {
    console.log("\n\n\n");
    console.log(`Finished Reading File ${data.length}`);
    //console.log(data); //in case we want to print out the contents of the file
    console.log("\n\n\n");
});

ReadStream {
  _readableState: 
   ReadableState {
     objectMode: false,
     highWaterMark: 65536,
     buffer: BufferList { head: null, tail: null, length: 0 },
     length: 0,
     pipes: null,
     pipesCount: 0,
     flowing: true,
     ended: false,
     endEmitted: false,
     reading: false,
     sync: true,
     needReadable: false,
     emittedReadable: false,
     readableListening: false,
     resumeScheduled: true,
     defaultEncoding: 'utf8',
     ranOut: false,
     awaitDrain: 0,
     readingMore: false,
     decoder: 
      StringDecoder {
        encoding: 'utf8',
        fillLast: [Function: utf8FillLast],
        lastNeed: 0,
        lastTotal: 0,
        lastChar: <Buffer af 0a 00 00> },
     encoding: 'UTF-8' },
  readable: true,
  domain: null,
  _events: 
   { end: [ [Function], [Function] ],
     data: [ [Object], [Function] ] },
  _eventsCount: 2,
  _maxListeners: undefined,
  path: './longTextFile.txt',
  fd: null,
  flags: 'r',
  mode: 438,
  start: undef





Started Reading File




  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65536 |  chunk: 65

### Writable streams



On the other side of the stream coin we have writable streams. The writable stream is used to write the data chunks that are going to be read by the readable streams.

In [10]:
var fs = require("fs");
var data = 'This is some data to be written into a writable Stream';

// Create a writable stream
var writerStream = fs.createWriteStream('output.txt');

// Write the data to stream with encoding to be utf8
writerStream.write(data,'UTF8');

// Mark the end of file
writerStream.end();

// Handle stream events --> finish, and error
writerStream.on('finish', function() {
    console.log("Write completed.");
});

writerStream.on('error', function(err){
   console.log(err.stack);
});

console.log("Program Ended");

Program Ended


undefined

Write completed.


### Piping streams

Piping is a mechanism where we provide the output of one stream as the input to another stream. It is normally used to get data from one stream and to pass the output of that stream to another stream.

In [11]:
var fs = require("fs");

// Create a readable stream
var readerStream = fs.createReadStream('input.txt');

// Create a writable stream
var writerStream = fs.createWriteStream('output.txt');

// Pipe the read and write operations
// read input.txt and write data to output.txt
readerStream.pipe(writerStream);

console.log("Program Ended");

Program Ended


undefined

-----------------------------------------------------

## Exercises

1. Create a script that appends a string to a non-empty text file.
2. This exercise requires you to do a bit of research on the `node.js` `crypto` module. Use a `readStream` to read the content of a text file and create a SHA1 digest of the file contents and output the SHA1 digest to the terminal.
3. Create a program that creates 2 buffers and compares their content. If both buffers are equal, your program should concatenate the buffers, otherwise do nothing.